In [ ]:
source("/root/workspace/code/midas/preprocess/utils.R")
setwd("/root/workspace/code/midas/")

base_dir <- "data/raw/atac+adt/asap/BM"
frag_path <- pj(base_dir, "GSM4732140_Human_BoneMarrow_hg38_fragments.tsv.gz")
adt_path <- pj(base_dir, "GSM4732141_Human_BoneMarrow_ADT.tsv.gz")
hto_path <- pj(base_dir, "GSM4732142_Human_BoneMarrow_HTO.tsv.gz")

output_dir <- pj(base_dir, "seurat")
mkdir(output_dir, remove_old = T)

In [ ]:
# ADT
# load data
adt_counts <- t(read.table(file = adt_path, sep = "\t", header = TRUE, row.names = 1))
adt <- gen_adt(adt_counts)
VlnPlot(adt, c("nCount_adt"), pt.size = 0.001, ncol = 1, log = T) + NoLegend()
adt
# QC
adt <- subset(adt, subset = nCount_adt > 150 & nCount_adt < 10000)
VlnPlot(adt, c("nCount_adt"), pt.size = 0.001, ncol = 1, log = T) + NoLegend()
adt

In [ ]:
# # HTO
# # load data
# hto_counts <- t(read.table(file = hto_path, sep = "\t", header = TRUE, row.names = 1))
# hto_counts <- hto_counts[, intersect(colnames(hto_counts), colnames(adt_counts))]
# hto <- CreateSeuratObject(counts = hto_counts, assay = "hto")
# # remove doublets
# hto <- NormalizeData(hto, assay = "hto", normalization.method = "CLR", margin = 1)
# hto <- HTODemux(hto, assay = "hto", positive.quantile = 0.995)
# table(hto$hto_classification.global)
# hto <- subset(hto, subset = hto_classification.global != "Doublet")
# hto

In [ ]:
# ATAC
atac <- gen_atac(frag_path)
VlnPlot(atac, c("nCount_atac", "nucleosome_signal", "TSS.enrichment"),
        pt.size = 0.001, ncol = 3, log = T) + NoLegend()
atac
# QC
atac <- subset(atac, subset =
    nCount_atac > 500 & nCount_atac < 3e4 &
    nucleosome_signal < 3 &
    TSS.enrichment > 2
)
atac <- subset(atac, features = rownames(atac)[rowSums(atac$atac@counts > 0) > 5])
VlnPlot(atac, c("nCount_atac", "nucleosome_signal", "TSS.enrichment"),
        pt.size = 0.001, ncol = 3, log = F) + NoLegend()
atac

In [19]:
# Get intersected cells satisfying QC metrics of all modalities
cell_ids <- Reduce(intersect, list(colnames(atac), colnames(adt)))
atac <- subset(atac, cells = cell_ids)
adt <- subset(adt, cells = cell_ids)
atac
adt

An object of class Seurat 
150502 features across 10671 samples within 1 assay 
Active assay: atac (150502 features, 0 variable features)

An object of class Seurat 
238 features across 10671 samples within 1 assay 
Active assay: adt (238 features, 0 variable features)

In [ ]:
# preprocess and save data
preprocess(output_dir, atac = atac, adt = adt)